In [ ]:
# Loc cong tac phan tich ma tran

In [5]:
from __future__ import print_function
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

class MF(object):
    def __init__(self, Y, K, lam = 0.1, Xinit = None, Winit = None, learning_rate = 0.5, max_iter = 1000, print_every = 100):
        self.Y = Y # represents the utility matrix
        self.K = K
        self.lam = lam # regularization parameter
        self.learning_rate = learning_rate # for gradient descent
        self.max_iter = max_iter # maximum number of iterations
        self.print_every = print_every # print loss after each a few iters
        self.n_users = int(np.max(Y[:, 0])) + 1
        self.n_items = int(np.max(Y[:, 1])) + 1
        self.n_ratings = Y.shape[0] # number of known ratings
        self.X = np.random.randn(self.n_items, K) if Xinit is None else Xinit
        self.W = np.random.randn(K, self.n_items) if Winit is None else Winit

        self.b = np.random.randn(self.n_items) # items biases
        self.d = np.random.randn(self.n_users) # users biases

    def loss(self):
        L = 0
        for i in range(self.n_ratings):
            # user_id, item_id, rating
            n, m, rating = int(self.Y[i, 0]), int(self.Y[i, 1]), self.Y[i, 2]

            L += 0.5 * (self.X[m].dot(self.W[:, n]) + self.b[m] + self.d[n] - rating) ** 2
        
        L /= self.n_ratings
        # dont ever forget regularization
        return L + 0.5 * self.lam * (np.sum(self.X ** 2) + np.sum(self.W ** 2))

    def updateXb(self):
        for m in range(self.n_items):
            # get all users who rated item m and corresponding ratings
            ids = np.where(self.Y[:, 1] == m)[0]
            users_ids = self.Y[ids, 0].astype(np.int32)
            ratings = self.Y[ids, 2]
            Wm, dm = self.W[:, users_ids], self.d[users_ids]
            for i in range(30): # 30 iterations for each sub problem
                xm = self.X[m]
                error = xm.dot(Wm) + self.b[m] + dm - ratings
                grad_xm = error.dot(Wm.T) / self.n_ratings + self.lam * xm
                grad_bm = np.sum(error) / self.n_ratings

                # gradient descent
                self.X[m] -= self.learning_rate * grad_xm.reshape(-1)
                self.b[m] -= self.learning_rate * grad_bm
    def updateWd(self):
        for n in range(self.n_users):
            # get all items rated by user n and corresponding ratings
            ids = np.where(self.Y[:, 0] == n)[0]
            items_ids = self.Y[ids, 1].astype(np.int32)
            # print(items_ids)
            ratings = self.Y[ids, 2]
            Xn, bn = self.X[items_ids], self.b[items_ids]
            for i in range(30): # 30 iteration for each sub problem
                wn = self.W[:, n]
                error = Xn.dot(wn) + bn + self.d[n] - ratings
                # print(error.shape, wn.shape, Xn.shape, bn.shape)
                grad_wn = Xn.T.dot(error) / self.n_ratings + self.lam * wn
                grad_dn = np.sum(error) / self.n_ratings

                # gradient descent
                self.W[:, n] -= self.learning_rate * grad_wn.reshape(-1)
                self.d[n] -= self.learning_rate * grad_dn

    def fit(self):
        for it in range(self.max_iter):
            self.updateWd()
            self.updateXb()
            if (it + 1) % self.print_every == 0:
                rmse_train = self.evaluate_RMSE(self.Y)
                print("iter = %d, loss = %.4f, RMSE train = %.4f" % (it + 1, self.loss(), rmse_train))

    def pred(self, u, i):
        # predict the rating of user u for item i
        u, i = int(u), int(i)
        pred = self.X[i, :].dot(self.W[:, u]) + self.b[i] + self.d[u]

        return max(0, min(5, pred))
    
    def evaluate_RMSE(self, rate_test):
        n_tests = rate_test.shape[0]
        SE = 0
        for n in range(n_tests):
            pred = self.pred(rate_test[n, 0], rate_test[n, 1])
            SE += (pred - rate_test[n, 2]) ** 2
        
        RMSE = np.sqrt(SE / n_tests)

        return RMSE

In [3]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_base = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols)
ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols)

rate_train = np.array(ratings_base)
rate_test = np.array(ratings_test)

rate_train[:, :2] -= 1 # since index start from 0
rate_test[:, :2] -= 1 # since index start from 0

In [6]:
rs = MF(rate_train, K = 50, lam = .01, print_every = 5, learning_rate = 50, max_iter = 30)
rs.fit()

RMSE = rs.evaluate_RMSE(rate_test)
print("Matrix Factorization CF, RMSE = ", RMSE)

iter = 5, loss = 185.1903, RMSE train = 0.9433
iter = 10, loss = 185.1668, RMSE train = 0.9181
iter = 15, loss = 185.1626, RMSE train = 0.9135
iter = 20, loss = 185.1613, RMSE train = 0.9121
iter = 25, loss = 185.1606, RMSE train = 0.9114
iter = 30, loss = 185.1603, RMSE train = 0.9110
Matrix Factorization CF, RMSE =  0.9629150922320201
